# Traffic Crash Data Cleaning

- Removing unwanted features/columns
- Handling missing values
- Standardizing inputs



In [27]:
# Import required libraries
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import warnings
import os
import sys

# Suppress warnings but keep important ones
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('default', category=DeprecationWarning)

# 1. Load and Inspect Data

In [28]:
df = pd.read_csv('../data/raw/all_crashes.csv')

print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample of first few rows:")
display(df.head())

print("\nDataset Info:")
df.info()

Dataset Shape: (660505, 40)

Columns: ['geom', 'Num_Occupants', 'Num_MotorVehicles', 'Fatalities_calc', 'SuspectedSeriousInjury_calc', 'NonMotorist_calc', 'Aggregated_Non_Motorist_txt', 'CityTownship', 'CountyNameTxt', 'Region', 'Zipcode_txtCARTO', 'TribalLand_txtCARTO', 'InjuryLevel', 'ExceedingSpeedLimitInd_boolCARTO', 'Seatbelt_boolCARTO', 'DistractedDrivingCde_boolCARTO', 'AlcoholSuspectedCde_txtCARTO', 'DrugSuspectedCde_txtCARTO', 'WeatherCde_txtCARTO', 'SurfaceConditionCde_txtCARTO', 'RdwyTypeCde_txtCARTO', 'RelativeLocIntersectCde', 'PostedSpeedNbr_binCARTO', 'WorkZoneInd_txtCARTO', 'DateOfIncident', 'AgencyIdNbr_txtCARTO', 'Injuries_calc', 'LatDec', 'LongDec', 'RdwyNameTxt', 'LocalCodeTxt', 'doi_date', 'doi_time', 'doi_dow_txtCARTO', 'HitAndRunInd_txtCARTO', 'VehicleMakeModelYearTxt_list', 'VehicleTypeCde_txtCARTO_list', 'doi_hour_txtCARTO', 'CrashType', 'CrashDetail']

Sample of first few rows:


,geom,Num_Occupants,Num_MotorVehicles,Fatalities_calc,SuspectedSeriousInjury_calc,NonMotorist_calc,Aggregated_Non_Motorist_txt,CityTownship,CountyNameTxt,Region,...,LocalCodeTxt,doi_date,doi_time,doi_dow_txtCARTO,HitAndRunInd_txtCARTO,VehicleMakeModelYearTxt_list,VehicleTypeCde_txtCARTO_list,doi_hour_txtCARTO,CrashType,CrashDetail
0,POINT(-93.66935666 44.92911086),1,1,0,0,0,Motorist Only,MOUND,HENNEPIN,Metro,...,24010265,"September 30, 2024",01:00:00 PM,Monday,"No, Did not Leave Scene",2015 FORD ESCAPE,Sport Utility Vehicle,1:00 PM - 1:59 PM,Non-Collision,Immersion (Partial or Full)
1,POINT(-93.31948042 43.69167691),2,2,0,0,0,Motorist Only,BANCROFT,FREEBORN,Southeast,...,24104389,"September 30, 2024",06:41:00 AM,Monday,"No, Did not Leave Scene","2017 FORD FIESTA, 2021 KENWORTH TRUCK","Passenger Car, Medium / Heavy Trucks (More tha...",6:00 AM - 6:59 AM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport
2,POINT(-96.34675035 43.60859205),1,1,0,0,0,Motorist Only,BEAVER CREEK,ROCK,South Central,...,24000700,"September 30, 2024",04:17:00 AM,Monday,"No, Did not Leave Scene",2003 CHEVROLET 1500 SILVERADO LTZ,Pickup,4:00 AM - 4:59 AM,Collision w/ Fixed Object,Cable Median Barrier
3,POINT(-93.29405308 44.86186167),2,2,0,0,0,Motorist Only,BLOOMINGTON,HENNEPIN,Metro,...,24511549,"September 30, 2024",06:39:00 AM,Monday,"Yes, Driver or Car and Driver Left Scene","2014 CHEVROLET CRUZE, 2020 JEEP (POST 1988) GR...","Passenger Car, Sport Utility Vehicle",6:00 AM - 6:59 AM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport
4,POINT(-93.37451935 44.86024945),4,4,0,0,0,Motorist Only,BLOOMINGTON,HENNEPIN,Metro,...,24511554,"September 30, 2024",07:36:00 AM,Monday,"No, Did not Leave Scene","2013 TOYOTA RAV4, 2018 VOLVO VN, 2012 DODGE DU...","Sport Utility Vehicle, Medium / Heavy Trucks (...",7:00 AM - 7:59 AM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660505 entries, 0 to 660504
Data columns (total 40 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   geom                              660505 non-null  object 
 1   Num_Occupants                     660505 non-null  int64  
 2   Num_MotorVehicles                 660505 non-null  int64  
 3   Fatalities_calc                   660505 non-null  int64  
 4   SuspectedSeriousInjury_calc       660505 non-null  int64  
 5   NonMotorist_calc                  660505 non-null  int64  
 6   Aggregated_Non_Motorist_txt       660505 non-null  object 
 7   CityTownship                      659750 non-null  object 
 8   CountyNameTxt                     660505 non-null  object 
 9   Region                            660505 non-null  object 
 10  Zipcode_txtCARTO                  644445 non-null  float64
 11  TribalLand_txtCARTO               109

In [29]:
# Convert the 'geom' column from WKT to geometry objects
df['geometry'] = df['geom'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [30]:
gdf['DateOfIncident'] = pd.to_datetime(gdf['DateOfIncident'])
gdf['date'] = gdf['DateOfIncident'].dt.date
gdf['time'] = gdf['DateOfIncident'].dt.time
gdf['hour'] = gdf['DateOfIncident'].dt.hour
gdf['day_of_week'] = gdf['DateOfIncident'].dt.day_name()
gdf['month'] = gdf['DateOfIncident'].dt.month_name()

# Show the temporal columns
print("Sample of temporal features:")
display(gdf[['DateOfIncident', 'date', 'time', 'hour', 'day_of_week', 'month']].head())

Sample of temporal features:


,DateOfIncident,date,time,hour,day_of_week,month
0,2024-09-30 13:00:00,2024-09-30,13:00:00,13,Monday,September
1,2024-09-30 06:41:00,2024-09-30,06:41:00,6,Monday,September
2,2024-09-30 04:17:00,2024-09-30,04:17:00,4,Monday,September
3,2024-09-30 06:39:00,2024-09-30,06:39:00,6,Monday,September
4,2024-09-30 07:36:00,2024-09-30,07:36:00,7,Monday,September


Kept information about location, surrounding environment, weather/surface conditions, temporal for further exploration

In [53]:
# Select the columns that could possibly be useful for prediction
selected_cols = ['geom', 'CountyNameTxt', 'Region', 
                'WeatherCde_txtCARTO', 'SurfaceConditionCde_txtCARTO', 'RdwyTypeCde_txtCARTO', 'WorkZoneInd_txtCARTO',
                'PostedSpeedNbr_binCARTO', 'DateOfIncident']
new_df = gdf[selected_cols].copy()

new_df.head()

,geom,CountyNameTxt,Region,WeatherCde_txtCARTO,SurfaceConditionCde_txtCARTO,RdwyTypeCde_txtCARTO,WorkZoneInd_txtCARTO,PostedSpeedNbr_binCARTO,DateOfIncident
0,POINT(-93.66935666 44.92911086),HENNEPIN,Metro,Clear,Water (Standing or Moving),No Input,No,NaN,2024-09-30 13:00:00
1,POINT(-93.31948042 43.69167691),FREEBORN,Southeast,Clear,Dry,Interstate Trunk Highway - ISTH,No,56 - 70 MPH,2024-09-30 06:41:00
2,POINT(-96.34675035 43.60859205),ROCK,South Central,Clear,Dry,Interstate Trunk Highway - ISTH,No,56 - 70 MPH,2024-09-30 04:17:00
3,POINT(-93.29405308 44.86186167),HENNEPIN,Metro,Clear,Dry,Interstate Trunk Highway - ISTH,No,41 - 55 MPH,2024-09-30 06:39:00
4,POINT(-93.37451935 44.86024945),HENNEPIN,Metro,Clear,Dry,Interstate Trunk Highway - ISTH,No,56 - 70 MPH,2024-09-30 07:36:00


# 2. Handle Missing Values

In [54]:
missing_values = new_df.isnull().sum()
print("Missing Values in Each Column:")
print(missing_values)

Missing Values in Each Column:
geom                                0
CountyNameTxt                       0
Region                              0
WeatherCde_txtCARTO                 0
SurfaceConditionCde_txtCARTO        0
RdwyTypeCde_txtCARTO                0
WorkZoneInd_txtCARTO                0
PostedSpeedNbr_binCARTO         29344
DateOfIncident                      0
dtype: int64


Find values that are null, missing, or unknown and mark them as 'unknown' for uniformity

In [68]:
for col in new_df.columns:
    print(f"{col}: {new_df[col].unique()}")

geom: ['POINT(-93.66935666 44.92911086)' 'POINT(-93.31948042 43.69167691)'
 'POINT(-96.34675035 43.60859205)' ... 'POINT(-93.08816289 44.97885924)'
 'POINT(-93.22959614 44.97268876)' 'POINT(-94.72390659 45.37884918)']
CountyNameTxt: ['HENNEPIN' 'FREEBORN' 'ROCK' 'DAKOTA' 'WASHINGTON' 'RAMSEY' 'GRANT'
 'OLMSTED' 'LINCOLN' 'LE SUEUR' 'WINONA' 'WABASHA' 'SIBLEY' 'CARVER'
 'GOODHUE' 'SAINT LOUIS' 'WRIGHT' 'MILLE LACS' 'ANOKA' 'BECKER' 'DOUGLAS'
 'CROW WING' 'MCLEOD' 'LYON' 'BROWN' 'COTTONWOOD' 'MOWER' 'SCOTT' 'SWIFT'
 'STEARNS' 'RICE' 'STEELE' 'BLUE EARTH' 'NICOLLET' 'SHERBURNE' 'BENTON'
 'MEEKER' 'CHISAGO' 'MORRISON' 'POPE' 'OTTER TAIL' 'WADENA' 'KANDIYOHI'
 'FARIBAULT' 'CARLTON' 'REDWOOD' 'LAKE' 'BELTRAMI' 'ISANTI' 'PINE' 'TODD'
 'JACKSON' 'POLK' 'ITASCA' 'LAC QUI PARLE' 'COOK' 'MURRAY' 'WASECA'
 'MARTIN' 'CHIPPEWA' 'WATONWAN' 'NOBLES' 'CLAY' 'AITKIN' 'HOUSTON'
 'RENVILLE' 'DODGE' 'MAHNOMEN' 'NORMAN' 'HUBBARD' 'PENNINGTON' 'BIG STONE'
 'YELLOW MEDICINE' 'FILLMORE' 'KOOCHICHING' 'MARSHALL

In [ ]:

def standardize_unknown_values(df):
    # Define unknown variations once
    unknown_variations = {
        'unknown', 'did not describe', 'not described', 'missing',
        'other', 'unspecified', 'n/a', 'na', 'none', 'no input',
        'not known at time of the crash'
    }
    
    # Process each column in the dataframe
    for col in df.columns:
        # Handle numeric columns separately to avoid string operations
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna('unknown')
            continue
            
        # For string columns, create a mask for all variations of unknown
        mask = df[col].isna()  # Get null values
        
        # Convert to string and lowercase for comparison
        str_series = df[col].astype(str).str.lower().str.strip()
        mask |= str_series.isin(unknown_variations)
        
        # Apply the replacement where mask is True
        df.loc[mask, col] = 'unknown'
    
    return df

In [71]:
# Apply the standardization function
new_df = standardize_unknown_values(new_df)

# Calculate statistics for unknown values
total_records = len(new_df)
unknown_stats = {}

for col in new_df.columns:
    if col != 'geom' and col != 'DateOfIncident':  # Skip non-categorical columns
        unknown_count = (new_df[col] == 'unknown').sum()
        unknown_percentage = (unknown_count / total_records) * 100
        unknown_stats[col] = {
            'unknown_count': unknown_count,
            'unknown_percentage': unknown_percentage
        }

# Display the results
print(f"Total records in dataset: {total_records:,}\n")
print("Unknown Values Statistics:")
print("-" * 60)
print(f"{'Column':<30} {'Count':>10} {'Percentage':>12}")
print("-" * 60)
for col, stats in unknown_stats.items():
    print(f"{col:<30} {stats['unknown_count']:>10,} {stats['unknown_percentage']:>11.2f}%")

Total records in dataset: 660,505

Unknown Values Statistics:
------------------------------------------------------------
Column                              Count   Percentage
------------------------------------------------------------
CountyNameTxt                           0        0.00%
Region                                  0        0.00%
WeatherCde_txtCARTO                 5,134        0.78%
SurfaceConditionCde_txtCARTO        4,734        0.72%
RdwyTypeCde_txtCARTO                7,299        1.11%
WorkZoneInd_txtCARTO                1,285        0.19%
PostedSpeedNbr_binCARTO            29,344        4.44%


In [98]:
print(new_df.describe(include='all').to_string())
print("\nHeaders of Each Column:", new_df.columns.tolist())

                                           geom CountyNameTxt  Region WeatherCde_txtCARTO SurfaceConditionCde_txtCARTO             RdwyTypeCde_txtCARTO WorkZoneInd_txtCARTO PostedSpeedNbr_binCARTO       DateOfIncident
count                                    660505        660505  660505              660505                       660505                           660505               660505                  660505               660505
unique                                   634965            87       8                   9                           12                               37                    3                       8               570863
top     POINT(-89.506587842973 48.004777071176)      HENNEPIN   Metro               Clear                          Dry  County State Aid Highway - CSAH                   No             41 - 55 MPH  2022-11-14 08:00:00
freq                                      16055        187505  407999              428548                       448215          

# 3. Save Cleaned Dataset
Save the cleaned dataset with standardized values and proper documentation.

In [97]:
# Create processed data directory if it doesn't exist
import os
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Save the cleaned dataset
cleaned_file_path = os.path.join(processed_dir, 'cleaned_crashes.csv')
new_df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to: {cleaned_file_path}")


Cleaned dataset saved to: ../data/processed\cleaned_crashes.csv


# Dataset Analysis and Recommendations

## Current Features
1. **Spatial Information**:
   - `geom`: Location data (geometric points)
   - `CountyNameTxt`: County level location
   - `Region`: Regional divisions

2. **Environmental Conditions**:
   - `WeatherCde_txtCARTO`: Weather conditions
   - `SurfaceConditionCde_txtCARTO`: Road surface state
   - `RdwyTypeCde_txtCARTO`: Road type
   - `WorkZoneInd_txtCARTO`: Work zone presence
   - `PostedSpeedNbr_binCARTO`: Speed limit information

3. **Temporal Information**:
   - `DateOfIncident`: Full timestamp
   - Derived features: hour, day_of_week, month


## Recommendations 

1. **Additional Data Sources**:
   - Weather data integration for more detailed conditions
   - Traffic volume data to understand exposure
   - Road infrastructure database
   - Census data for demographic/population context

2. **Feature Engineering**:
   - Create time-based risk factors
   - Develop location-based risk scores
   - Calculate proximity to key facilities
   - Narrow down roadway types to categories like highway, city, county

3. **Spatial Analysis**: 
   - Include population density information
   - Consider road network characteristics

4. **Temporal Patterns**:
   - Add holiday indicators
   - Include special event information
   - Consider seasonal patterns
